# AWS S3 Operations using Boto3

In this notebook, we will perform several operations on AWS S3 using Boto3. 

These operations include:

- creating an S3 bucket
- writing an object
- listing objects
- reading the contents of an object
- deleting an object
- updating an object
- and handling errors for non-existent objects. 
  
We will also demonstrate how to write files with different prefixes and query files by prefix.

## For this notebook to run...

You will need to have these libraries installed

- `boto3`
- `boto3-stubs[s3]`
- `rich`
- `ipykernel`

The recommended approach is to set up your `pyproject.toml` like so:

```toml
[project]
...
dependencies = ['importlib-metadata; python_version<"3.8"', "boto3"]

[project.optional-dependencies]
stubs = ["boto3-stubs[s3]"]
notebooks = ["jupyterlab", "ipykernel", "rich"]
...
dev = ["cloud-course-project[test,release,static-code-qa,stubs,notebooks]"]
```

So that `pip install --editable './[dev]'` will install all the necessary dependencies into your venv.

## Instructions for setting up autocompletion in Jupyter Notebooks in VS Code

1. Install the development dependencies:
```sh
pip install --editable './[dev]'
```
2. Select the notebook kernel and point it to your virtual environment:
```sh
which python
```
3. Select the VS Code Python interpreter and point it to your virtual environment:
```sh
which python
```
4. Reload the VS Code window (`Ctrl/Cmd + Shift + P` > `Developer: Reload Window`)


In [1]:
# Import necessary libraries
import boto3
from botocore.exceptions import ClientError
from uuid import uuid4  # randomly generated string
from rich import print  # pretty printing
import os
from typing import Optional

try:
    from mypy_boto3_s3 import S3Client
except ImportError:
    print("mypy_boto3_s3 not installed")

In [2]:
# define constants

# Set the profile and region for the AWS SDK (boto3) to use
os.environ["AWS_PROFILE"] = "cloud-course"
os.environ["AWS_REGION"] = "us-east-1"

# Create a session using the specified profile and region
S3_CLIENT: "S3Client" = boto3.client("s3")

BUCKET_NAME = f"cloud-course-bucket-{str(uuid4())[:4]}"

# Single example object
EXAMPLE_OBJECT_KEY = "example/object/file.txt"
EXAMPLE_OBJECT_CONTENT = "This is a test object."

# Multiple example objects
EXAMPLE_OBJECTS = [
    ("example-a/object/file1.txt", "This is a test object."),
    ("example-a/object/file2.txt", "This is another test object."),
    ("example-a/object/file3.txt", "This is yet another test object."),
    ("example-b/object/file1.txt", "This is a test object."),
    ("example-b/object/file2.txt", "This is another test object."),
    ("example-b/object/file3.txt", "This is yet another test object."),
]

print(f"{BUCKET_NAME=}")
print(f"{EXAMPLE_OBJECT_KEY=}")
print(f"{EXAMPLE_OBJECT_CONTENT=}")

BUCKET_NAME='cloud-course-bucket-1a50'

EXAMPLE_OBJECT_KEY='example/object/file.txt'

EXAMPLE_OBJECT_CONTENT='This is a test object.'

## Create a bucket

Here, we create an S3 bucket. Anything you can do in the AWS console, you can do programatically!

In [3]:
try:
    from mypy_boto3_s3.type_defs import CreateBucketOutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def create_bucket(bucket_name: str) -> Optional["CreateBucketOutputTypeDef"]:
    """
    Create an S3 bucket.

    :param bucket_name: Name of the bucket to create
    :type bucket_name: str
    """
    response = S3_CLIENT.create_bucket(Bucket=bucket_name)
    return response


# Create the bucket
response = create_bucket(bucket_name=BUCKET_NAME)
print(f"Bucket '{BUCKET_NAME}' created successfully.")
print(response)

Bucket 'cloud-course-bucket-1a50' created successfully.

{
    'ResponseMetadata': {
        'RequestId': 'ZNPCDZQJ0D2ETTPC',
        'HostId': 'oxIuOy28jZp+DPI1eRHxd/+6l8YVtpDy1cN9Z8p9y9cYrIfe+EfRCoEpNJGW8jeimgPauyU8Ukw=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'oxIuOy28jZp+DPI1eRHxd/+6l8YVtpDy1cN9Z8p9y9cYrIfe+EfRCoEpNJGW8jeimgPauyU8Ukw=',
            'x-amz-request-id': 'ZNPCDZQJ0D2ETTPC',
            'date': 'Mon, 10 Jun 2024 05:20:22 GMT',
            'location': '/cloud-course-bucket-1a50',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'Location': '/cloud-course-bucket-1a50'
}

## Write an object to the bucket

In this exercise, you will write an object to the S3 bucket.

"S3 Paths" are URLs of the form `s3://bucket-name/key`. The `key` is the path to the object in the bucket. For example, `s3://my-bucket-name/my-key` refers to the object with key `my-key` in the bucket `my-bucket-name`.

Some examples:

| Path | Bucket | Object Key |
| --- | --- | --- |
| `s3://my-bucket/images/profile.jpeg` | `my-bucket` | `images/profile.jpeg` |
| `s3://my-bucket/data/2021/01/01/data.csv` | `my-bucket` | `data/2021/01/01/data.csv` |
| `s3://my-bucket/file.json` | `my-bucket` | `file.json` |


In [4]:
try:
    from mypy_boto3_s3.type_defs import PutObjectOutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def write_text_object_to_s3(
    bucket_name: str,
    object_key: str,
    object_content: str,
) -> Optional["PutObjectOutputTypeDef"]:
    """
    Write an object to an S3 bucket.

    :param bucket_name: Name of the bucket to write to
    :param object_key: Key of the object to write
    :param object_content: Content of the object to write
    :return: Response from the put_object call
    """
    response = S3_CLIENT.put_object(Bucket=bucket_name, Key=object_key, Body=object_content)
    return response


# Write the single example to S3
response = write_text_object_to_s3(
    bucket_name=BUCKET_NAME,
    object_key=EXAMPLE_OBJECT_KEY,
    object_content=EXAMPLE_OBJECT_CONTENT,
)
print(response)

# Write the rest of the examples to S3
for object_key, object_content in EXAMPLE_OBJECTS:
    print(f"Writing object to path 's3://{BUCKET_NAME}/{object_key}'")
    write_text_object_to_s3(
        bucket_name=BUCKET_NAME,
        object_key=object_key,
        object_content=object_content,
    )

{
    'ResponseMetadata': {
        'RequestId': 'MK7XMYYZD3SHH5X9',
        'HostId': 'KvD8xgM2fMQ64fpHq1yQshz7hCuGE0w9jvb05PWByocANZvabz/TIX9Tfl5GsccxZhTpcSytYU0=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'KvD8xgM2fMQ64fpHq1yQshz7hCuGE0w9jvb05PWByocANZvabz/TIX9Tfl5GsccxZhTpcSytYU0=',
            'x-amz-request-id': 'MK7XMYYZD3SHH5X9',
            'date': 'Mon, 10 Jun 2024 05:20:27 GMT',
            'x-amz-server-side-encryption': 'AES256',
            'etag': '"fac2cbcd7b7417c0325922b689019c65"',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'ETag': '"fac2cbcd7b7417c0325922b689019c65"',
    'ServerSideEncryption': 'AES256'
}

Writing object to path 's3://cloud-course-bucket-1a50/example-a/object/file1.txt'

Writing object to path 's3://cloud-course-bucket-1a50/example-a/object/file2.txt'

Writing object to path 's3://cloud-course-bucket-1a50/example-a/object/file3.txt'

Writing object to path 's3://cloud-course-bucket-1a50/example-b/object/file1.txt'

Writing object to path 's3://cloud-course-bucket-1a50/example-b/object/file2.txt'

Writing object to path 's3://cloud-course-bucket-1a50/example-b/object/file3.txt'

## Read the content of an object

In this exercise, you will read the content of an object from the S3 bucket.

In [5]:
from botocore.response import StreamingBody


def read_text_object_from_s3(bucket_name: str, object_key: str, verbose: bool = False) -> str | None:
    """
    Read the content of an object from an S3 bucket.

    :param bucket_name: Name of the bucket to read from
    :param object_key: Key of the object to read
    :return: Content of the object
    """
    response = S3_CLIENT.get_object(Bucket=bucket_name, Key=object_key)

    # Note, we need to read the bytestream from the blob and choose how we wish
    # to interpret the bytes. In this case, we interpret them as utf-8 encoded text.
    content_streaming_body: StreamingBody = response["Body"]
    content: str = content_streaming_body.read().decode("utf-8")
    return content


# Read the content of the example object
content = read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
if content:
    print(f"Content of object '{EXAMPLE_OBJECT_KEY}':\n'{content}'")

Content of object 'example/object/file.txt':
'This is a test object.'

## Exercise #1 - List objects in the bucket

What if there were more than 1,000 objects in the bucket? How would you list all of them?

Hints
- Look into ["boto3 paginators"](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/paginators.html#creating-paginators)
- OR Look into the [boto3 "resource API"](https://boto3.amazonaws.com/v1/documentation/api/latest/guide/resources.html#resources), a powerful, but potentially less performant, object-oriented way
  of interacting with AWS resources via boto3
- OR consider using "Continuation Tokens" with the `list_objects_v2` method.

In [6]:
try:
    from mypy_boto3_s3.type_defs import ListObjectsV2OutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def list_all_object_keys_in_bucket(bucket_name: str) -> list[str]:
    """
    List all objects in an S3 bucket.

    :param bucket_name: Name of the bucket to list objects from
    :return: List of object keys
    """
    # fill this out...
    paginator =S3_CLIENT.get_paginator('list_objects_v2')

    page_iterator = paginator.paginate(Bucket=bucket_name)

    return page_iterator


# List all objects in the bucket
objects: "ListObjectsV2OutputTypeDef" = list_all_object_keys_in_bucket(BUCKET_NAME)
if objects:
    print("Objects in bucket:")
    for obj in objects:
        print(f" - {obj}\n")

Objects in bucket:

- {'ResponseMetadata': {'RequestId': '7DSS5HFXTHJ4ND1Q', 'HostId': 
'KOfS0GdWdiHS35XfIY2M1st2gSt/30RpKmd72kOQ+BEge+cuxQwATiVlBl3Fn6D9BSF4WJ6Ve9M=', 'HTTPStatusCode': 200, 
'HTTPHeaders': {'x-amz-id-2': 'KOfS0GdWdiHS35XfIY2M1st2gSt/30RpKmd72kOQ+BEge+cuxQwATiVlBl3Fn6D9BSF4WJ6Ve9M=', 
'x-amz-request-id': '7DSS5HFXTHJ4ND1Q', 'date': 'Mon, 10 Jun 2024 05:20:40 GMT', 'x-amz-bucket-region': 
'us-east-1', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 
'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'example-a/object/file1.txt', 'LastModified': 
datetime.datetime(2024, 6, 10, 5, 20, 28, tzinfo=tzutc()), 'ETag': '"fac2cbcd7b7417c0325922b689019c65"', 'Size': 
22, 'StorageClass': 'STANDARD'}, {'Key': 'example-a/object/file2.txt', 'LastModified': datetime.datetime(2024, 6, 
10, 5, 20, 28, tzinfo=tzutc()), 'ETag': '"df3d558947aab328a1678715276eaa75"', 'Size': 28, 'StorageClass': 
'STANDARD'}, {'Key': 'example-a/object/file3.txt', 'LastModified': datetime.datetime(2024, 6, 10, 5, 20, 29, 
tzinfo=tzutc()), 'ETag': '"1f2b86f2d8f4e6ac910e31b4b05a593f"', 'Size': 32, 'StorageClass': 'STANDARD'}, {'Key': 
'example-b/object/file1.txt', 'LastModified': datetime.datetime(2024, 6, 10, 5, 20, 29, tzinfo=tzutc()), 'ETag': 
'"fac2cbcd7b7417c0325922b689019c65"', 'Size': 22, 'StorageClass': 'STANDARD'}, {'Key': 
'example-b/object/file2.txt', 'LastModified': datetime.datetime(2024, 6, 10, 5, 20, 30, tzinfo=tzutc()), 'ETag': 
'"df3d558947aab328a1678715276eaa75"', 'Size': 28, 'StorageClass': 'STANDARD'}, {'Key': 
'example-b/object/file3.txt', 'LastModified': datetime.datetime(2024, 6, 10, 5, 20, 30, tzinfo=tzutc()), 'ETag': 
'"1f2b86f2d8f4e6ac910e31b4b05a593f"', 'Size': 32, 'StorageClass': 'STANDARD'}, {'Key': 'example/object/file.txt', 
'LastModified': datetime.datetime(2024, 6, 10, 5, 20, 27, tzinfo=tzutc()), 'ETag': 
'"fac2cbcd7b7417c0325922b689019c65"', 'Size': 22, 'StorageClass': 'STANDARD'}], 'Name': 'cloud-course-bucket-1a50',
'Prefix': '', 'MaxKeys': 1000, 'EncodingType': 'url', 'KeyCount': 7}

## Exercise #2 - Update the object (overwrite with new content)

In this exercise, you will update the content of an existing object in the S3 bucket.

In [7]:
new_content = "This is updated content."

# the "PUT object" command is an upsert, so it will overwrite existing files
response = write_text_object_to_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY, new_content)
print(response)

# Read the updated content of the object
updated_content = read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
if updated_content:
    print(f"Content of updated object '{EXAMPLE_OBJECT_KEY}':\n{updated_content}")

{
    'ResponseMetadata': {
        'RequestId': '5RCQECDM3D6CEMEE',
        'HostId': '/Sctph9THFndkV12u/p4ECTpaZ9rTlV/y9g7I2hMQSIvrmEh6aaSSei2yQfu7g/lGilrcm4TDcQ=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': '/Sctph9THFndkV12u/p4ECTpaZ9rTlV/y9g7I2hMQSIvrmEh6aaSSei2yQfu7g/lGilrcm4TDcQ=',
            'x-amz-request-id': '5RCQECDM3D6CEMEE',
            'date': 'Mon, 10 Jun 2024 05:20:45 GMT',
            'x-amz-server-side-encryption': 'AES256',
            'etag': '"c5a01ee632fac2f88da9e0aa5a3d8cdd"',
            'server': 'AmazonS3',
            'content-length': '0'
        },
        'RetryAttempts': 0
    },
    'ETag': '"c5a01ee632fac2f88da9e0aa5a3d8cdd"',
    'ServerSideEncryption': 'AES256'
}

Content of updated object 'example/object/file.txt':
This is updated content.

## Exercise #3 - Delete the object from the bucket

In this exercise, you will delete an object from the S3 bucket.

In [8]:
try:
    from mypy_boto3_s3.type_defs import DeleteObjectOutputTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def delete_object_from_s3(
    bucket_name: str,
    object_key: str,
) -> Optional["DeleteObjectOutputTypeDef"]:
    """
    Delete an object from an S3 bucket.

    :param bucket_name: Name of the bucket to delete the object from
    :param object_key: Key of the object to delete
    :return: Response from the delete_object call
    """
    # fill this out...
    response = S3_CLIENT.delete_object(Bucket=bucket_name, Key=object_key)
    return response


# Delete the example object
response = delete_object_from_s3(bucket_name=BUCKET_NAME, object_key=EXAMPLE_OBJECT_KEY)
print(response)

{
    'ResponseMetadata': {
        'RequestId': 'DDH3BBYVJD4HK9FR',
        'HostId': '4O2oOVntAuI1xMxOwGYcAAZ7caNNMldjlaA7JdnCGsachULnnSy5DtrqWnUcScN0dz3D55Tmah0=',
        'HTTPStatusCode': 204,
        'HTTPHeaders': {
            'x-amz-id-2': '4O2oOVntAuI1xMxOwGYcAAZ7caNNMldjlaA7JdnCGsachULnnSy5DtrqWnUcScN0dz3D55Tmah0=',
            'x-amz-request-id': 'DDH3BBYVJD4HK9FR',
            'date': 'Mon, 10 Jun 2024 05:20:49 GMT',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    }
}

## Exercise #4 - Try to read a non-existent object

In this exercise, you will attempt to read a non-existent object from the S3 bucket. 

Catch, suppress, and print the `ClientError` exception that is raised when you try to read a non-existent object.

In [9]:
print(f"Trying to read the deleted object at 's3://{BUCKET_NAME}/{EXAMPLE_OBJECT_KEY}' ...")

delete_object_from_s3(bucket_name=BUCKET_NAME, object_key=EXAMPLE_OBJECT_KEY)

try:
    # try to read the deleted object
    response = read_text_object_from_s3(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
    print(response)
except ClientError as err:
    assert "NoSuchKey" in str(err)

Trying to read the deleted object at 's3://cloud-course-bucket-1a50/example/object/file.txt' ...

## Exercise #5 - Try to delete a non-existent object

**Note:** the result of calling `s3_client.delete_object` on a non-existent object is not what you might expect. It succeeds whether or not there exists an object with the given key.

**Note:** The HTTP status code `204` means `No Content`. Or in other words, the request is successful
but there was nothing to delete.

In [10]:
non_existant_object_key = EXAMPLE_OBJECT_KEY + "_non_existent"

# delete the non-existent object
# ...
response = delete_object_from_s3(bucket_name=BUCKET_NAME, object_key=non_existant_object_key)

print(response)

{
    'ResponseMetadata': {
        'RequestId': 'ZQK4XWYZXQ140BNF',
        'HostId': 'g9dPxQ12NlFTbbQTJJ+8hi1iAghvv0lxuW0CRmoKBDcV4dXqU3HxDaOBKdA84MEHznaPV16WGGk=',
        'HTTPStatusCode': 204,
        'HTTPHeaders': {
            'x-amz-id-2': 'g9dPxQ12NlFTbbQTJJ+8hi1iAghvv0lxuW0CRmoKBDcV4dXqU3HxDaOBKdA84MEHznaPV16WGGk=',
            'x-amz-request-id': 'ZQK4XWYZXQ140BNF',
            'date': 'Mon, 10 Jun 2024 05:21:00 GMT',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    }
}

## Exercise #6 - Error handling when deleting an object

In this exercise, you will implement error handling to raise a `FileNotFoundError` if you try to delete a non-existent object, i.e., one that has already been deleted or was never written.

Hint, the `s3_client.head_object(...)` method raises an error with status code `404 - File Not Found`
if no file exists for the given object key.

In [11]:
HTTP_FILE_NOT_FOUND_ERROR_CODE = "404"


class S3FileNotFoundError(Exception):
    """Raise this exception when an object at a given path is not found in S3."""


def delete_object_or_error_if_not_exists(bucket_name: str, object_key: str) -> None:
    """
    Delete an object from an S3 bucket with error handling for non-existent objects.

    :param bucket_name: Name of the bucket to delete the object from
    :param object_key: Key of the object to delete

    :raises S3FileNotFoundError: if no object exists at the given path
    :raises ClientError: if an unexpected error occurs when using S3 that is not due to file not found
    """
    # fill this out...
    S3_CLIENT.head_object(Bucket=bucket_name, Key=object_key)
    response = S3_CLIENT.delete_object(Bucket=bucket_name, Key=object_key)
    return response

# Try to delete the non-existent example object with error handling
try:
    delete_object_or_error_if_not_exists(BUCKET_NAME, EXAMPLE_OBJECT_KEY)
except (ClientError, S3FileNotFoundError) as err:
    print(err)

An error occurred (404) when calling the HeadObject operation: Not Found

## Exercise #7 - List objects in the bucket to confirm they were deleted in the previous exercises

In this exercise, you will list all objects in the S3 bucket to confirm that the object has been deleted.

In [35]:
# List all objects in the bucket
objects = list_all_object_keys_in_bucket(BUCKET_NAME)

if objects:
    print("Objects in bucket:")
    for obj in objects:
        print(f" - {obj}")
else:
    print("Bucket is empty.")

Objects in bucket:

- {'ResponseMetadata': {'RequestId': '3K7WMAPB4CFTZNP4', 'HostId': 
'dZruBy8YrHS/OTXbrjjv1DcTSWbqKCDrqtj+7nx6aAK7ojWxUTRKbUxxsxDwrtZwxa3eqtN6jBE=', 'HTTPStatusCode': 200, 
'HTTPHeaders': {'x-amz-id-2': 'dZruBy8YrHS/OTXbrjjv1DcTSWbqKCDrqtj+7nx6aAK7ojWxUTRKbUxxsxDwrtZwxa3eqtN6jBE=', 
'x-amz-request-id': '3K7WMAPB4CFTZNP4', 'date': 'Sun, 09 Jun 2024 17:27:22 GMT', 'x-amz-bucket-region': 
'us-east-1', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 
'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'example-a/object/file1.txt', 'LastModified': 
datetime.datetime(2024, 6, 9, 17, 22, 38, tzinfo=tzutc()), 'ETag': '"fac2cbcd7b7417c0325922b689019c65"', 'Size': 
22, 'StorageClass': 'STANDARD'}, {'Key': 'example-a/object/file2.txt', 'LastModified': datetime.datetime(2024, 6, 
9, 17, 22, 39, tzinfo=tzutc()), 'ETag': '"df3d558947aab328a1678715276eaa75"', 'Size': 28, 'StorageClass': 
'STANDARD'}, {'Key': 'example-a/object/file3.txt', 'LastModified': datetime.datetime(2024, 6, 9, 17, 22, 40, 
tzinfo=tzutc()), 'ETag': '"1f2b86f2d8f4e6ac910e31b4b05a593f"', 'Size': 32, 'StorageClass': 'STANDARD'}, {'Key': 
'example-b/object/file1.txt', 'LastModified': datetime.datetime(2024, 6, 9, 17, 22, 40, tzinfo=tzutc()), 'ETag': 
'"fac2cbcd7b7417c0325922b689019c65"', 'Size': 22, 'StorageClass': 'STANDARD'}, {'Key': 
'example-b/object/file2.txt', 'LastModified': datetime.datetime(2024, 6, 9, 17, 22, 41, tzinfo=tzutc()), 'ETag': 
'"df3d558947aab328a1678715276eaa75"', 'Size': 28, 'StorageClass': 'STANDARD'}, {'Key': 
'example-b/object/file3.txt', 'LastModified': datetime.datetime(2024, 6, 9, 17, 22, 41, tzinfo=tzutc()), 'ETag': 
'"1f2b86f2d8f4e6ac910e31b4b05a593f"', 'Size': 32, 'StorageClass': 'STANDARD'}], 'Name': 'cloud-course-bucket-c33d',
'Prefix': '', 'MaxKeys': 1000, 'EncodingType': 'url', 'KeyCount': 6}

## Exercise 8 - List objects by prefix

In this exercise, you will list objects in the S3 bucket by prefix.

In [18]:
def list_all_objects_in_bucket_by_prefix(bucket_name: str, prefix: str) -> list[str]:
    """
    List objects in an S3 bucket by prefix.

    :param bucket_name: Name of the bucket to list objects from
    :param prefix: Prefix to filter objects by
    :return: List of object keys
    """
    # fill this out...
    response = S3_CLIENT.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    return response


# List objects by prefix
prefix = "example-a/"
objects_by_prefix = list_all_objects_in_bucket_by_prefix(BUCKET_NAME, prefix)
if objects_by_prefix:
    print(f"Objects with prefix '{prefix}':")
    for obj in objects_by_prefix:
        print(f" - {obj}")
else:
    print(f"No objects found with prefix '{prefix}'.")

Objects with prefix 'example-a/':

- ResponseMetadata

- IsTruncated

- Name

- Prefix

- MaxKeys

- EncodingType

- KeyCount

## Exercise 9 - Delete a bucket, no matter what!

In this exercise, you will delete the S3 bucket. Your bucket may have objects in it. Does that matter?

Write a function to delete your bucket. 

***Be careful to point it at the right bucket using the right
AWS credentials--or you might delete the wrong bucket!***

In [12]:
import boto3
from botocore.exceptions import ClientError

try:
    from mypy_boto3_s3.type_defs import EmptyResponseMetadataTypeDef
except ImportError:
    print("boto3-stubs[s3] not installed")


def delete_bucket(bucket_name: str) -> Optional["EmptyResponseMetadataTypeDef"]:
    """
    Delete an S3 bucket, including all its objects.

    If the bucket does not exist, no error is raised.

    :param bucket_name: Name of the bucket to delete
    :return: Response from the delete_bucket call or None if there is no bucket.
    """
    # fill this out...
    default_kwargs = {
        "Bucket": bucket_name,
    }
    next_token = ""
    while next_token is not None:
        updated_kwargs = default_kwargs.copy()
        if next_token != "":
            updated_kwargs["ContinuationToken"] = next_token

        list_obj_response = S3_CLIENT.list_objects_v2(**updated_kwargs)

        for obj in list_obj_response.get("Contents"):
            S3_CLIENT.delete_object(Bucket = bucket_name, Key = obj.get("Key"))

        next_token = list_obj_response.get("NextContinuationToken")
        
    response = S3_CLIENT.delete_bucket(Bucket=bucket_name)
    return response


response = delete_bucket(BUCKET_NAME)
print(response)

{
    'ResponseMetadata': {
        'RequestId': '3MM73Y66DGKEB6GA',
        'HostId': 'cgk6dWmTYuFFsywWtN/zuZlZ2F0MSxzZYUcdTIkzrum7Mow+F7yuSjxLUOlGvAiNG/CoUOvfowk=',
        'HTTPStatusCode': 204,
        'HTTPHeaders': {
            'x-amz-id-2': 'cgk6dWmTYuFFsywWtN/zuZlZ2F0MSxzZYUcdTIkzrum7Mow+F7yuSjxLUOlGvAiNG/CoUOvfowk=',
            'x-amz-request-id': '3MM73Y66DGKEB6GA',
            'date': 'Mon, 10 Jun 2024 06:21:56 GMT',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    }
}

## (Optional) Bonus Exercise #1 - Recursively upload a local directory to S3

In [23]:
from pathlib import Path
import pathlib
from typing import Generator


def recursively_upload_dir_to_bucket(
    local_dir_fpath: str | Path,
    bucket_name: str,
    target_root_prefix_in_bucket: str = "",
):
    """
    Recurse through a local directory and upload all files to S3 under a target prefix.

    The object keys within the bucket should be the relative paths of the files within the local directory.

    Example:

    path/to/local_dir/
    ├── file1.txt
    ├── file2.txt
    └── subdir
        └── file3.txt

    Would be uploaded to

    s3://bucket-name/target_root_prefix_in_bucket/
    ├── file1.txt
    ├── file2.txt
    └── subdir/
        └── file3.txt
    """
    # fill this out...
    path = pathlib.Path(local_dir_fpath)
    for p in path.rglob("*"):
        if p.is_file():
            key = Path.joinpath(Path(target_root_prefix_in_bucket),p.relative_to(path))
            S3_CLIENT.upload_file(Filename=str(p), Bucket=bucket_name, Key=str(key))

# create a test dir locally with sample files
test_dir = Path("test_dir")
test_dir.mkdir(parents=True, exist_ok=True)
(test_dir / "file1.txt").write_text("This is file 1.")
(test_dir / "file2.txt").write_text("This is file 2.")
(test_dir / "subdir").mkdir(parents=True, exist_ok=True)
(test_dir / "subdir" / "file3.txt").write_text("This is file 3.")

# clean up the bucket
try:
    delete_bucket(BUCKET_NAME)
except:
    print(f"Bucket {BUCKET_NAME} already deleted.")

create_bucket(bucket_name=BUCKET_NAME)

recursively_upload_dir_to_bucket(
    bucket_name=BUCKET_NAME,
    local_dir_fpath=test_dir,
    target_root_prefix_in_bucket="test-root-dir/",
)

# List all objects in the bucket at the test target root
objects = list_all_objects_in_bucket_by_prefix(
    bucket_name=BUCKET_NAME,
    prefix="test-root-dir/",
)
print(objects)

{
    'ResponseMetadata': {
        'RequestId': '3GX250CHGY98CY5T',
        'HostId': 'tUtuxrK8+tuwncze5hNRbZQiTd9jf4jYVAWZwdApc/JDEXfA21wc0d+ADuFdZ1l5NT8wIugezVo=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'tUtuxrK8+tuwncze5hNRbZQiTd9jf4jYVAWZwdApc/JDEXfA21wc0d+ADuFdZ1l5NT8wIugezVo=',
            'x-amz-request-id': '3GX250CHGY98CY5T',
            'date': 'Mon, 10 Jun 2024 07:48:34 GMT',
            'x-amz-bucket-region': 'us-east-1',
            'content-type': 'application/xml',
            'transfer-encoding': 'chunked',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    },
    'IsTruncated': False,
    'Contents': [
        {
            'Key': 'test-root-dir/file1.txt',
            'LastModified': datetime.datetime(2024, 6, 10, 7, 48, 32, tzinfo=tzutc()),
            'ETag': '"cfb2a3740155c041d2c3e13ad1d66644"',
            'Size': 15,
            'StorageClass': 'STANDARD'
        },
        {
            'Key': 'test-root-dir/file2.txt',
            'LastModified': datetime.datetime(2024, 6, 10, 7, 48, 33, tzinfo=tzutc()),
            'ETag': '"500d703f270d4bc034e159480c83d329"',
            'Size': 15,
            'StorageClass': 'STANDARD'
        },
        {
            'Key': 'test-root-dir/subdir/file3.txt',
            'LastModified': datetime.datetime(2024, 6, 10, 7, 48, 33, tzinfo=tzutc()),
            'ETag': '"e06c939b5c696e61ad74870b905903cd"',
            'Size': 15,
            'StorageClass': 'STANDARD'
        }
    ],
    'Name': 'cloud-course-bucket-1a50',
    'Prefix': 'test-root-dir/',
    'MaxKeys': 1000,
    'EncodingType': 'url',
    'KeyCount': 3
}

## (Optional) Bonus Exercise #2 - Rename a "folder" in an S3 bucket

S3 is a key-value store for blobs of bytes. There is no way to "rename" a "folder" in S3
without changing the key names of each object in the "folder".

To really feel the weight and implications of this fact, go through this exercise 🤣

Ultimately, you have to copy each object one at a time, and delete the old object. For large buckets, e.g.
data lakes with millions of files, this is a slow process.

In [27]:
from gc import get_objects


def rename_folder_in_bucket(
    bucket_name: str,
    old_folder_prefix: str,
    new_folder_prefix: str,
):
    """
    Rename a "folder" in an S3 bucket.

    Example:

    Given the following structure in S3:

    s3://bucket-name/<old_folder_prefix>/
    ├── file1.txt
    ├── file2.txt
    └── subdir/
        └── file3.txt

    After renaming

    s3://bucket-name/<new_folder_prefix>/
    ├── file1.txt
    ├── file2.txt
    └── subdir/
        └── file3.txt

    :param bucket_name: Name of the S3 bucket
    :param source_folder: Source "folder" path in the bucket
    :param destination_folder: Destination "folder" path in the bucket
    """
    # fill this out...
    bucket_list = S3_CLIENT.list_objects_v2(Bucket=bucket_name, Prefix=old_folder_prefix)
    for obj in bucket_list.get("Contents"):
        print(obj)
        old_key = obj.get("Key")
        value = obj.get("StreamingBody")
        S3_CLIENT.delete_object(Bucket=bucket_name, Key=old_key)
        new_key = old_key.removeprefix(old_folder_prefix)
        new_key = new_folder_prefix + new_key
        S3_CLIENT.put_object(Bucket=bucket_name, Key=new_key, Body = value)



# def move_object_in_bucket(bucket_name: str, source_key: str, destination_key: str):
#     """
#     Move an object within an S3 bucket by copying to the new key and deleting the old key.

#     :param bucket_name: Name of the S3 bucket
#     :param source_key: Source key of the object to move
#     :param destination_key: Destination key of the object
#     """
#     # fill this out...


"""Test the rename_folder_in_bucket function."""


def upload_file_to_bucket(
    local_fpath: str | Path,
    bucket_name: str,
    target_key_in_bucket: str,
):
    """
    Upload a file to an S3 bucket.

    :param local_fpath: Local file path to upload
    :param bucket_name: Name of the bucket to upload the file to
    :param target_key_in_bucket: Key to upload the file to in the bucket
    """
    local_fpath = Path(local_fpath)
    with open(local_fpath, "rb") as file:
        S3_CLIENT.put_object(Bucket=bucket_name, Key=target_key_in_bucket, Body=file)


# Create test objects in the source folder
test_source_folder = "nested/source-folder01/"
test_dest_folder = "nested/destination-folder01/"

create_bucket(bucket_name=BUCKET_NAME)

# Upload test files to source folder
upload_file_to_bucket(
    local_fpath="test_dir/file1.txt", bucket_name=BUCKET_NAME, target_key_in_bucket=test_source_folder + "file1.txt"
)
upload_file_to_bucket(
    local_fpath="test_dir/file2.txt", bucket_name=BUCKET_NAME, target_key_in_bucket=test_source_folder + "file2.txt"
)
upload_file_to_bucket(
    local_fpath="test_dir/subdir/file3.txt",
    bucket_name=BUCKET_NAME,
    target_key_in_bucket=test_source_folder + "subdir/file3.txt",
)

# Rename the source folder to the destination folder
rename_folder_in_bucket(
    bucket_name=BUCKET_NAME,
    old_folder_prefix=test_source_folder,
    new_folder_prefix=test_dest_folder,
)

# List all objects in the destination folder
objects = list_all_objects_in_bucket_by_prefix(bucket_name=BUCKET_NAME, prefix=test_dest_folder)
print(objects)

{
    'Key': 'nested/source-folder01/file1.txt',
    'LastModified': datetime.datetime(2024, 6, 10, 7, 52, 3, tzinfo=tzutc()),
    'ETag': '"cfb2a3740155c041d2c3e13ad1d66644"',
    'Size': 15,
    'StorageClass': 'STANDARD'
}

ParamValidationError: Parameter validation failed:
Invalid type for parameter Body, value: None, type: <class 'NoneType'>, valid types: <class 'bytes'>, <class 'bytearray'>, file-like object